**https://github.com/dcf21/planisphere/blob/master/bright_stars_process.py**

In [2]:
# bright_stars_process.py
# -*- coding: utf-8 -*-
#
# The python script in this file makes the various parts of a model planisphere.
#
# Copyright (C) 2014-2023 Dominic Ford <https://dcford.org.uk/>
#
# This code is free software; you can redistribute it and/or modify it under
# the terms of the GNU General Public License as published by the Free Software
# Foundation; either version 2 of the License, or (at your option) any later
# version.
#
# You should have received a copy of the GNU General Public License along with
# this file; if not, write to the Free Software Foundation, Inc., 51 Franklin
# Street, Fifth Floor, Boston, MA  02110-1301, USA

# ----------------------------------------------------------------------------

"""
This script takes the Yale Bright Star Catalogue, and formats it into a python list. It also adds the names of objects.
"""

import re


def fetch_bright_star_list():
    """
    Read the Yale Bright Star Catalogue from disk, and return it as a list of stars.

    :return:
        Dictionary
    """
    # Astronomical unit, in metres
    AU = 1.49598e11

    # Light year, in metres
    LYR = 9.4605284e15

    # Build a dictionary of stars, indexed by HD number
    stars = {}

    # Convert three-letter abbreviations of Greek letters into UTF-8
    greek_alphabet = {'Alp': '\u03b1', 'Bet': '\u03b2', 'Gam': '\u03b3', 'Del': '\u03b4', 'Eps': '\u03b5',
                      'Zet': '\u03b6', 'Eta': '\u03b7', 'The': '\u03b8', 'Iot': '\u03b9', 'Kap': '\u03ba',
                      'Lam': '\u03bb', 'Mu': '\u03bc', 'Nu': '\u03bd', 'Xi': '\u03be', 'Omi': '\u03bf',
                      'Pi': '\u03c0', 'Rho': '\u03c1', 'Sig': '\u03c3', 'Tau': '\u03c4', 'Ups': '\u03c5',
                      'Phi': '\u03c6', 'Chi': '\u03c7', 'Psi': '\u03c8', 'Ome': '\u03c9'}

    # Superscript numbers which we may place after Greek letters to form the Flamsteed designations of stars
    star_suffices = {'1': '\u00B9', '2': '\u00B2', '3': '\u00B3'}

    # Look up the common names of bright stars
    star_names = {}
    with open("raw_data/bright_star_names.dat", "rt") as f_in:
        for line in f_in:
            # Ignore blank lines and comment lines
            if (len(line) < 5) or (line[0] == '#'):
                continue
            # Catalog is indexed by the HR number of each star in the first column
            bs_num = int(line[0:4])

            # The second column is the name of the star, with underscores in the place of spaces
            name = line[5:]
            star_names[bs_num] = re.sub(' ', '_', name.strip())

    # Loop through the Yale Bright Star Catalog, line by line
    bs_num = 0
    with open("raw_data/bright_star_catalog.dat", "rt") as f_in:
        for line in f_in:
            # Ignore blank lines and comment lines
            if (len(line) < 100) or (line[0] == '#'):
                continue

            # Counter used too calculated the bright star number -- i.e. the HR number -- of each star
            bs_num += 1
            try:
                # Read the Henry Draper (i.e. HD) number for this star
                hd = int(line[25:31])

                # Read the right ascension of this star (J2000)
                ra_hrs = float(line[75:77])
                ra_min = float(line[77:79])
                ra_sec = float(line[79:82])

                # Read the declination of this star (J2000)
                dec_neg = (line[83] == '-')
                dec_deg = float(line[84:86])
                dec_min = float(line[86:88])
                dec_sec = float(line[88:90])

                # Read the V magnitude of this star
                mag = float(line[102:107])
            except ValueError:
                continue

            # Look up the Bayer number of this star, if one exists
            star_num = -1
            try:
                star_num = int(line[4:7])
            except ValueError:
                pass

            # Render a unicode string containing the name, Flamsteed designation, and Bayer designation for this star
            name_bayer = name_bayer_full = name_english = name_flamsteed_full = "-"

            # Look up the Greek letter (Flamsteed designation) of this star
            greek = line[7:10].strip()

            # Look up the abbreviation of the constellation this star is in
            const = line[11:14].strip()

            # Some stars have a suffix after the Flamsteed designation, e.g. alpha-1, alpha-2, etc.
            greek_letter_suffix = line[10]
            if greek in greek_alphabet:
                name_bayer = greek_alphabet[greek]
                if greek_letter_suffix in star_suffices:
                    name_bayer += star_suffices[greek_letter_suffix]
                name_bayer_full = '{}-{}'.format(name_bayer, const)
            if star_num > 0:
                name_flamsteed_full = '{}-{}'.format(star_num, const)

            # See if this is a star with a name
            if bs_num in star_names:
                name_english = star_names[bs_num]

            # Turn RA and Dec from sexagesimal units into decimal
            ra = (ra_hrs + ra_min / 60 + ra_sec / 3600) / 24 * 360
            dec = (dec_deg + dec_min / 60 + dec_sec / 3600)
            if dec_neg:
                dec = -dec

            # Build a dictionary is stars, indexed by HD number
            stars[hd] = [ra, dec, mag, name_bayer, name_bayer_full, name_english, name_flamsteed_full]

    hd_numbers = list(stars.keys())
    hd_numbers.sort()

    return {
        'stars': stars,
        'hd_numbers': hd_numbers
    }



**Convert the dictionary into a pandas dataframe fro readability**

In [239]:
import pandas as pd


# Fetch the data and create DataFrame
data = fetch_bright_star_list()
df = pd.DataFrame.from_dict(data['stars'], orient='index',
                            columns=['RA', 'Dec', 'Magnitude', 'Bayer', 'Full Bayer', 'English Name', 'Flamsteed'])

# Display the DataFrame 
display(df)

num_stars = len(data['stars']) # Number of stars in the dictionary
print(f"Number of stars processed: {num_stars}") # veriyf Number of stars in the dictionary






,RA,Dec,Magnitude,Bayer,Full Bayer,English Name,Flamsteed
3,1.287500,45.229167,6.70,-,-,-,-
6,1.262500,-0.503056,6.29,-,-,-,-
28,1.333333,-5.707500,4.61,-,-,-,33-Psc
87,1.425000,13.396111,5.51,-,-,-,86-Peg
123,1.566667,58.436667,5.96,-,-,-,-
...,...,...,...,...,...,...,...
225233,1.125000,-72.897778,7.31,-,-,-,-
225239,1.220833,34.659722,6.12,-,-,-,-
225253,1.170833,-71.436944,5.59,-,-,-,-
225276,1.233333,26.648889,6.25,-,-,-,-


Number of stars processed: 9096


**Output the dataframe into a .mat file for matlab use**

In [ ]:
from scipy.io import savemat

# Convert DataFrame to a dictionary where the column names are keys
mat_data = {col: df[col].values for col in df.columns}

savemat('bright_stars.mat', mat_data)

**Define a function to "page" through the dataframe**

In [224]:
def paginate_dataframe(df, page_size):
    """Simple pagination helper to go through the DataFrame."""
    last_row = 0
    while last_row < len(df):
        yield df.iloc[last_row:last_row + page_size]
        last_row += page_size

# Example usage
pager = paginate_dataframe(df, 25)  # Change 20 to however many rows you want per page


In [225]:
try:
    next_page = next(pager)
    print(next_page)
except StopIteration:
    print("You have reached the end of the DataFrame.")

           RA        Dec  Magnitude Bayer Full Bayer English Name Flamsteed
3    1.287500  45.229167       6.70     -          -            -         -
6    1.262500  -0.503056       6.29     -          -            -         -
28   1.333333  -5.707500       4.61     -          -            -    33-Psc
87   1.425000  13.396111       5.51     -          -            -    86-Peg
123  1.566667  58.436667       5.96     -          -            -         -
142  1.579167 -49.075000       5.70     -          -            -         -
144  1.608333  64.196111       5.59     -          -            -    10-Cas
166  1.650000  29.021389       6.13     -          -            -         -
203  1.708333 -23.107500       6.18     -          -            -         -
256  1.825000 -17.386389       6.19     -          -            -         -
315  1.933333  -2.548889       6.43     -          -            -         -
319  1.941667 -22.508889       5.94     -          -            -         -
344  2.01250